In [53]:
import numpy as np
import pandas as pd
from numpy.linalg import inv, matrix_rank
from numpy import matmul
from scipy.stats import f

In [54]:
# R dXK, d linear restrictions and K predictors. Rank(R) = d

In [214]:
alpha = 0.05
df1 = 1 # rank of R. df1
df2 = 192 # n-K
f.ppf(alpha, df1, df2) # critical value

0.003942433704199765

In [383]:
def sim_data(betas, rows = 100):
    xs = np.random.random((rows, len(betas)))
    y = (xs @ betas).reshape((rows,1)) +np.random.normal(0,1,(rows,1))
    return xs, y
    
    
def estimate_beta(xs, y):
    est = inv(xs.T @ xs) @xs.T @ y
    return est
    

def f_stat(R, betas, xs, r, y):
    d = matrix_rank(R)
    #print(d)
    n = len(xs)
    K = len(betas)
    e = y - xs @ betas
    #print(e.shape)
    upper = ((R @ betas - r).T @ inv(R @ inv(xs.T @ xs) @ R.T) @ (R@betas -r))/d
    #print(upper)
    sample_var = (e.T @ e)/(n-K)
    #print(sample_var)
    F = upper/sample_var
    return [F, d, n-K]


def make_R_r(*args, betas):
    R = np.zeros((len(args), len(betas)))
    for idx,arg in enumerate(args):
        R[idx, arg] = 1
    
    r = (R @ betas).reshape((len(args),1))
    return R, r
        

def test(F,df1, df2, alpha = 0.05):
    crit = f.ppf(1-alpha, df1, df2)
    if F <= crit:
        print('Accept H0')
    else:
        print('Reject H0')
    return crit



def sim(betas, R, r, res = True): # puts much of the functions together. *args are desired restrictions
    xs,y = sim_data(betas)
    beta_est = estimate_beta(xs,y)
    F, df1, df2 = f_stat(R, beta_est, xs, r, y)
    if res == True:
        crit = test(F, df1, df2)
        print(f'{'-'*60}')
        print(f'Critical value: {round(crit, 3)}')
        print(f'F: {round(F[0][0],3)}')
        print(f'beta estimates: {[n for n in beta_est]}')
        print(f'True betas: {[n for n in betas]}')
    else:
        crit = f.ppf(1-0.05, df1, df2)
    return crit, F
    

In [384]:
betas = np.array([1,5,3,8]).reshape((4,1))
R, r = make_R_r(1,3, [1,2,3], betas=betas)
crit, F = sim(betas, R, r)

Accept H0
------------------------------------------------------------
Critical value: 2.699
F: 1.961
beta estimates: [array([0.35698822]), array([5.3527284]), array([3.31394643]), array([8.22685863])]
True betas: [array([1]), array([5]), array([3]), array([8])]


In [393]:
n = 0
reject = 0
for i in range(1000):
    betas = np.array([1,5,3,8]).reshape((4,1))
    R, r = make_R_r(1,3, [1,2,3], betas=betas)
    crit, F = sim(betas, R, r, res=False)
    if F>crit:
        reject += 1
    else:
        pass
    n += 1
print(reject)
print(n)
print(f'Rejected {reject/n*100}% of simulations')
    
    

53
1000
Rejected 5.3% of simulations
